In [9]:
import redis
import yaml
import numpy as np
import random
import dask.array as da

In [4]:
with open('../settings.yaml') as f:
    sets = yaml.load(f, Loader=yaml.FullLoader)

In [5]:
r = redis.Redis(sets['redis']['host'], port=sets['redis']['port'], db=0)

In [18]:
#https://math.stackexchange.com/questions/2858815/understanding-formula-for-hyperplanes

In [6]:
H,W=100000,512
HPs = 1024 # number of hyperplanes, equals to number of bits in key

In [7]:
Y = da.random.normal(size=(H, W))

In [8]:
Y

dask.array<normal, shape=(100000, 512), dtype=float64, chunksize=(25000, 512), chunktype=numpy.ndarray>

In [11]:
idxes = np.empty([HPs, 2], dtype=np.int)
for i in range(HPs):
    idxes[i,:]=np.random.choice(a=H, size=2, replace=False)

In [37]:
a=Y[idxes[:,0]]
b=Y[idxes[:,1]]

In [38]:
b_a=b-a
a_norm2=da.linalg.norm(a, axis=1)
b_norm2=da.linalg.norm(b, axis=1)
c=b_norm2-a_norm2

In [39]:
X = da.random.normal(size=(1, W))

In [35]:
data = np.random.rand(H,W)

In [77]:
norms=1

In [21]:
a=np.random.rand(D)
a_norm2=np.dot(a,a)
b=np.random.rand(D)
b_norm2=np.dot(b,b)

In [22]:
for i in range(10):
    x=np.random.rand(D)
    print(np.dot((b-a),x)-0.5*(b_norm2-a_norm2))

-0.1710563516679867
0.09007390803510262
-0.3126568818384522
-0.6698706360217792
-0.37790388714794215
0.0135725719437913
0.06801615491527663
-0.4088421532486562
-0.17820569717080706
-0.5156129506644247
